In [1]:
import pandas as pd

In [2]:
# Data obtained from https://opendata.minneapolismn.gov/datasets/police-stop-data
df = pd.read_csv('./data/mnpls.csv')

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,OBJECTID,masterIncidentNumber,responseDate,reason,problem,callDisposition,citationIssued,personSearch,vehicleSearch,preRace,race,gender,lat,long,x,y,policePrecinct,neighborhood,lastUpdateDate
0,1,16-395258,2016/10/31 22:40:47+00,NaN,Suspicious Person (P),BKG-Booking,NaN,YES,NO,Black,Black,Male,44.979570,-93.272570,-1.038306e+07,5.618306e+06,1.0,Downtown West,2017/08/08 10:25:31+00
1,2,16-395296,2016/10/31 23:06:36+00,NaN,Traffic Law Enforcement (P),TAG-Tagged,NaN,NO,NO,Unknown,Black,Male,44.962689,-93.275921,-1.038343e+07,5.615650e+06,5.0,Steven's Square - Loring Heights,2017/08/08 10:26:13+00
2,3,16-395326,2016/10/31 23:20:54+00,NaN,Attempt Pick-Up (P),RFD-Refused,NaN,NO,NO,Unknown,Unknown,Unknown,45.024836,-93.288069,-1.038478e+07,5.625432e+06,4.0,Webber - Camden,2017/08/08 10:24:35+00
3,4,16-395328,2016/10/31 23:23:20+00,NaN,Suspicious Person (P),BKG-Booking,NaN,YES,NO,Black,Black,Male,44.946560,-93.247410,-1.038025e+07,5.613112e+06,3.0,Corcoran,2017/08/08 10:25:31+00
4,5,16-395333,2016/10/31 23:26:05+00,NaN,Suspicious Vehicle (P),GOA-Gone on Arrival,NaN,NO,NO,Other,Unknown,Male,44.906170,-93.255010,-1.038110e+07,5.606762e+06,3.0,Hale,2017/08/08 10:25:03+00


In [4]:
drop_cols = [
    "OBJECTID",
    "masterIncidentNumber",
    "problem",
    "callDisposition",
    "citationIssued",
    "personSearch",
    "vehicleSearch",
    "preRace",
    "race",
    "gender",
    "lastUpdateDate",
    "lat",
    "long",
    "x",
    "y",
]

In [5]:
df.drop(columns=drop_cols, inplace=True)

In [6]:
df.head()

,responseDate,reason,policePrecinct,neighborhood
0,2016/10/31 22:40:47+00,NaN,1.0,Downtown West
1,2016/10/31 23:06:36+00,NaN,5.0,Steven's Square - Loring Heights
2,2016/10/31 23:20:54+00,NaN,4.0,Webber - Camden
3,2016/10/31 23:23:20+00,NaN,3.0,Corcoran
4,2016/10/31 23:26:05+00,NaN,3.0,Hale


In [7]:
df["responseDate"] = pd.to_datetime(df.responseDate)

In [8]:
df.reason.value_counts()

Citizen / 9-1-1        46306
Moving Violation       42815
Investigative          29821
Equipment Violation    21287
Name: reason, dtype: int64

In [9]:
df = df[ df.reason == "Equipment Violation" ].drop(columns="reason")

In [10]:
df.head()

,responseDate,policePrecinct,neighborhood
18918,2017-03-20 14:17:41+00:00,4.0,Hawthorne
18957,2017-03-20 20:29:37+00:00,3.0,Ventura Village
18966,2017-03-20 21:56:04+00:00,4.0,Jordan
18967,2017-03-20 21:56:15+00:00,5.0,Tangletown
18982,2017-03-21 01:19:44+00:00,2.0,Marcy Holmes


In [11]:
df["month"] = df.responseDate.dt.month

In [12]:
df["day"] = df.responseDate.dt.day

In [13]:
df.head()

,responseDate,policePrecinct,neighborhood,month,day
18918,2017-03-20 14:17:41+00:00,4.0,Hawthorne,3,20
18957,2017-03-20 20:29:37+00:00,3.0,Ventura Village,3,20
18966,2017-03-20 21:56:04+00:00,4.0,Jordan,3,20
18967,2017-03-20 21:56:15+00:00,5.0,Tangletown,3,20
18982,2017-03-21 01:19:44+00:00,2.0,Marcy Holmes,3,21


In [14]:
df = df.loc[ (df.month == 7) & (df.day >= 1) & (df.day <=  7) ]

In [15]:
counts = df.groupby("neighborhood").responseDate.count()
counts.name = "counts"

In [16]:
counts = pd.DataFrame({'neighborhood': counts.index, 'incidents': counts.values}).sort_values('incidents', ascending=False)

In [19]:
counts.head(10)

,neighborhood,incidents
20,Hawthorne,21
53,Whittier,21
36,Near - North,18
31,Marcy Holmes,18
23,Jordan,15
21,Holland,13
11,Downtown West,12
37,Nicollet Island - East Bank,10
28,Lowry Hill East,8
54,Willard - Hay,8
